In [1]:
%matplotlib inline
import boto3
import botocore
import datetime
import matplotlib.pyplot as plt
import os.path
import xarray as xr

In [2]:
bucket = 'nasanex'

# AWS access / secret keys required
# s3 = boto3.resource('s3')
# bucket = s3.Bucket(era5_bucket)

# No AWS keys required
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

In [3]:
paginator = client.get_paginator('list_objects')
result = paginator.paginate(Bucket=bucket, Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

AVHRR/
CMIP5/
LOCA/
Landsat/
MAIAC/
MODIS/
NAIP/
NEX-DCP30/
NEX-GDDP/


In [7]:
keys = []
prefix = 'NEX-GDDP'
date = datetime.date(2018,1,1)
response = client.list_objects_v2(Bucket=bucket, Prefix=prefix)
response_meta = response.get('ResponseMetadata')

In [8]:
if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None:
        print("No objects are available for %s" % date.strftime('%B, %Y'))
    else:
        for obj in contents:
            keys.append(obj.get('Key'))
        print("There are %s objects available for %s\n--" % (len(keys), date.strftime('%B, %Y')))
        for k in keys:
            print(k)
else:
    print("There was an error with your request.")

There are 1000 objects available for January, 2018
--
NEX-GDDP/
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1950.json
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1950.nc
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1951.json
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1951.nc
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1952.json
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1952.nc
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1953.json
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1953.nc
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1954.json
NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1

In [13]:
metadata_file = 'pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1950.nc'
metadata_key = 'NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1950.nc'
client.download_file(bucket, metadata_key, metadata_file)
ds_meta = xr.open_dataset('pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1950.nc', decode_times=False)
ds_meta.info()

xarray.Dataset {
dimensions:
	lat = 720 ;
	lon = 1440 ;
	time = 365 ;

variables:
	float64 time(time) ;
		time:standard_name = time ;
		time:long_name = time ;
		time:axis = T ;
		time:calendar = proleptic_gregorian ;
		time:units = days since 0001-01-01 ;
		time:bounds = time_bnds ;
	float32 pr(time, lat, lon) ;
		pr:time = 711857.5 ;
		pr:standard_name = precipitation_flux ;
		pr:long_name = Precipitation ;
		pr:comment = at surface; includes both liquid and solid phases from all types of clouds (both large-scale and convective) ;
		pr:units = kg m-2 s-1 ;
		pr:cell_methods = time: mean ;
		pr:cell_measures = area: areacella ;
		pr:history = 2012-01-17T12:34:12Z altered by CMOR: replaced missing value flag (-1.07374e+09) with standard missing value (1e+20). ;
		pr:associated_files = baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_atmos_fx_ACCESS1-0_historical_r0i0p0.nc areacella: areacella_fx_ACCESS1-0_historical_r0i0p0.nc ;
	float32 lat(lat) ;
		lat:sta

In [17]:
metadata_file2 = 'pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1950.json'
metadata_key2 = 'NEX-GDDP/BCSD/historical/day/atmos/pr/r1i1p1/v1.0/pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1950.json'
client.download_file(bucket, metadata_key2, metadata_file2)

In [18]:
import pandas as pd

In [20]:
df = pd.read_json(metadata_file2)

In [21]:
df.head()

,Conventions,DOI,contact,creation_date,data,dimensions,disclaimer,downscalingModel,driving_data_tracking_ids,driving_experiment,...,realization,realm,references,resolution_id,source,table_id,title,tracking_id,variables,version
lat,CF-1.4,http://dx.doi.org/10.7292/W0MW2F2G,"Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bri...",2015-01-07T21:16:01Z,NaN,"{'length': 720, 'unlimited': False}",This data is considered provisional and subjec...,BCSD,N/A,piControl,...,1,atmos,"BCSD method: Thrasher et al., 2012, Hydrol. Ea...",0.25 degree,BCSD 2014,Table day (12 November 2010),ACCESS1-0 global downscaled NEX CMIP5 climate ...,893e10dc-110a-4983-b893-a374ceb5c60a,"{'axis': 'Y', 'dimensions': ['lat'], 'dtype': ...",1
lon,CF-1.4,http://dx.doi.org/10.7292/W0MW2F2G,"Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bri...",2015-01-07T21:16:01Z,NaN,"{'length': 1440, 'unlimited': False}",This data is considered provisional and subjec...,BCSD,N/A,piControl,...,1,atmos,"BCSD method: Thrasher et al., 2012, Hydrol. Ea...",0.25 degree,BCSD 2014,Table day (12 November 2010),ACCESS1-0 global downscaled NEX CMIP5 climate ...,893e10dc-110a-4983-b893-a374ceb5c60a,"{'axis': 'X', 'dimensions': ['lon'], 'dtype': ...",1
time,CF-1.4,http://dx.doi.org/10.7292/W0MW2F2G,"Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bri...",2015-01-07T21:16:01Z,NaN,"{'length': 365, 'unlimited': True}",This data is considered provisional and subjec...,BCSD,N/A,piControl,...,1,atmos,"BCSD method: Thrasher et al., 2012, Hydrol. Ea...",0.25 degree,BCSD 2014,Table day (12 November 2010),ACCESS1-0 global downscaled NEX CMIP5 climate ...,893e10dc-110a-4983-b893-a374ceb5c60a,"{'axis': 'T', 'bounds': 'time_bnds', 'calendar...",1
pr,CF-1.4,http://dx.doi.org/10.7292/W0MW2F2G,"Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bri...",2015-01-07T21:16:01Z,NaN,NaN,This data is considered provisional and subjec...,BCSD,N/A,piControl,...,1,atmos,"BCSD method: Thrasher et al., 2012, Hydrol. Ea...",0.25 degree,BCSD 2014,Table day (12 November 2010),ACCESS1-0 global downscaled NEX CMIP5 climate ...,893e10dc-110a-4983-b893-a374ceb5c60a,"{'_FillValue': '1e+20', 'associated_files': 'b...",1


{'axis': 'Y',
 'dimensions': ['lat'],
 'dtype': 'float32',
 'long_name': 'latitude',
 'standard_name': 'latitude',
 'units': 'degrees_north'}